In [1]:
# https://www.kaggle.com/datasets/gauravduttakiit/airbnb-berlin-price-prediction
# Dies ist eine Übung ohne Fragestellung. 
# Bei einer realen Aufgabenstellung wie "Schätze den Preis von einem gerade zu erstellenden Angebot" müssten andere Annahmen getroffen werden.
# Alles was auf den Host bezogen ist müsste entfernt werden, da dieser Faktor bei einem neuen Angebot nicht vorliegt. Folgerichtig müsste man sich die Frage stelle nwelchen Einfluss die Bewertungen auf den Preis haben und in wie weit dies sich gegenseitig beeinflussen. 
import pandas as pd 
test_df = pd.read_csv("test_airbnb_berlin.csv")
train_df = pd.read_csv("train_airbnb_berlin.csv")

columns_to_drop = []

# Voranalyse

## Reduktion

In [2]:
# listet ob spalten nur einen Wert haben
unique_count = train_df.nunique() == 1
# set null where more than one value is present and sort them out
cols_with_one_value = unique_count.where(lambda b : b == True).dropna().index.tolist()
columns_to_drop.extend(cols_with_one_value)
cols_with_one_value

['Listing Name', 'Country Code', 'Country', 'Business Travel Ready']

In [3]:
train_df = train_df.drop(columns=cols_with_one_value)
train_df.head(3)

,Listing ID,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,City,...,Last Review,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Price
0,19665213.0,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,...,2017-08-08,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,26.0
1,6436842.0,5302290.0,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,Berlin,...,2019-04-30,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,41.0
2,10559468.0,59151456.0,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,Berlin,...,2016-07-04,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,50.0


In [4]:
# spalten finden in denen ein Null vorhanden ist
has_nulls = train_df.isnull().any().where(lambda b : b == True).dropna().index.tolist()

In [5]:
# prozent von Nan-Werten in einer Spalte
percent_missing = train_df.isnull().sum() * 100 / len(train_df)
missing_value_df = pd.DataFrame({'column_name': train_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Listing ID,Listing ID,0.006373
Host ID,Host ID,0.000000
Host Name,Host Name,0.140199
Host Since,Host Since,0.133826
Host Response Time,Host Response Time,45.086668
Host Response Rate,Host Response Rate,45.086668
Is Superhost,Is Superhost,0.146572
neighbourhood,neighbourhood,0.000000
Neighborhood Group,Neighborhood Group,0.000000
City,City,0.006373


In [6]:
# alles verwerfen was über 10% lücken hat
over_10_percent = missing_value_df[missing_value_df['percent_missing'] > 10]['column_name'].tolist()
train_df = train_df.drop(columns=over_10_percent)
columns_to_drop.extend(over_10_percent)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15692 entries, 0 to 15691
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Listing ID          15691 non-null  float64
 1   Host ID             15692 non-null  float64
 2   Host Name           15670 non-null  object 
 3   Host Since          15671 non-null  object 
 4   Is Superhost        15669 non-null  object 
 5   neighbourhood       15692 non-null  object 
 6   Neighborhood Group  15692 non-null  object 
 7   City                15691 non-null  object 
 8   Postal Code         15463 non-null  object 
 9   Latitude            15692 non-null  float64
 10  Longitude           15692 non-null  float64
 11  Is Exact Location   15692 non-null  object 
 12  Property Type       15692 non-null  object 
 13  Room Type           15692 non-null  object 
 14  Accomodates         15692 non-null  object 
 15  Bathrooms           15678 non-null  object 
 16  Bedr

In [7]:
# semantisch unlogisches entfernen
# aus dem Hostnamen könnte man noch geschlecht oder nationalität ableiten
no_added_value = ['Listing ID', 'Host ID', 'Host Name', 'City', 'Postal Code']
train_df = train_df.drop(columns=no_added_value)
columns_to_drop.extend(no_added_value)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15692 entries, 0 to 15691
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Host Since          15671 non-null  object 
 1   Is Superhost        15669 non-null  object 
 2   neighbourhood       15692 non-null  object 
 3   Neighborhood Group  15692 non-null  object 
 4   Latitude            15692 non-null  float64
 5   Longitude           15692 non-null  float64
 6   Is Exact Location   15692 non-null  object 
 7   Property Type       15692 non-null  object 
 8   Room Type           15692 non-null  object 
 9   Accomodates         15692 non-null  object 
 10  Bathrooms           15678 non-null  object 
 11  Bedrooms            15687 non-null  object 
 12  Beds                15684 non-null  object 
 13  Guests Included     15692 non-null  object 
 14  Min Nights          15692 non-null  object 
 15  Reviews             15692 non-null  int64  
 16  Inst

## Aufräumen

In [8]:
# Schauen was ersetzt werden muss
train_df.isnull().any().where(lambda b : b == True).dropna().index.tolist()

['Host Since', 'Is Superhost', 'Bathrooms', 'Bedrooms', 'Beds', 'Price']

Host Since -> annahmne = heute

Is Superhost' -> ammahme = false

Bathrooms, Bedrooms, Beds -> annahme = 0

Price -> löschen

Alternativ noch Bewertungen drin lassen aber alles ohne Bewertungen löschen

## Anreicherung

In [9]:
# was kann abgeleitet werden aus den Daten? 
test_df = pd.read_csv("test_airbnb_berlin.csv") # neu laden da sich auch aus lückenhaften angeben noch was ableiten lassen kann
train_df.head(3)

,Host Since,Is Superhost,neighbourhood,Neighborhood Group,Latitude,Longitude,Is Exact Location,Property Type,Room Type,Accomodates,Bathrooms,Bedrooms,Beds,Guests Included,Min Nights,Reviews,Instant Bookable,Price
0,2016-01-20,f,Prenzlauer Berg,Pankow,52.54652,13.41792,t,Apartment,Private room,2,1.0,1.0,1.0,1,2,6,t,26.0
1,2013-04-07,f,Pankow,Pankow,52.56512,13.42214,t,Apartment,Entire home/apt,2,1.0,2.0,2.0,2,7,6,f,41.0
2,2016-02-07,f,Prenzlauer Berg,Pankow,52.54741,13.42521,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,1,1,2,f,50.0


Host Since -> Monaten an Vermietungszeit

Host Response -> ob er überhaupt jemals reagiert hat oder ordinal aufsteigend

Bewertungen -> ob überhapt schon jemand da war (steckt schon in der Anzahl der Bewertungen)

# Umsetzten und speichern

In [34]:
test_df = pd.read_csv("test_airbnb_berlin.csv")
train_df = pd.read_csv("train_airbnb_berlin.csv")

# Der Preis ist nur im Trainingsset
train_df.dropna(subset=['Price'])

def df_process(df_o):
    df = df_o.copy()
    # nulls füllen
    df['Host Since'] = df['Host Since'].fillna('2019-06-01')
    df['Is Superhost'] = df['Is Superhost'].fillna(False)
    df['Bathrooms'] = df['Bathrooms'].fillna(0)
    df['Bedrooms'] = df['Bedrooms'].fillna(0)
    df['Beds'] = df['Beds'].fillna(0)
    
    # die neusten Bewerungen sind aus Mai 2019 also nehmen wir den 01.06.2019 als Vergleichszeitpunkt an
    df['Host Since'] = pd.to_datetime(df['Host Since'], infer_datetime_format=True)
    df['Host Since'] = (pd.to_datetime('2019-06-01', infer_datetime_format=True) - pd.to_datetime(df['Host Since'], infer_datetime_format=True)) / pd.offsets.Day(1)
    df = df.rename(columns={'Host Since':'Host Since Days'})
    
    # wurde auf nachrichten reagiert
    df['Has Responses'] = df['Host Response Time'] is not None
    
    df = df.drop(columns=columns_to_drop)
    return df
    
test_df = df_process(test_df)
train_df = df_process(train_df)

C:\Users\sric\AppData\Local\Temp\ipykernel_6968\1651692499.py:17: UserWarning: Parsing '15-06-2014' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Host Since'] = pd.to_datetime(df['Host Since'], infer_datetime_format=True)
C:\Users\sric\AppData\Local\Temp\ipykernel_6968\1651692499.py:17: UserWarning: Parsing '29-04-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Host Since'] = pd.to_datetime(df['Host Since'], infer_datetime_format=True)
C:\Users\sric\AppData\Local\Temp\ipykernel_6968\1651692499.py:17: UserWarning: Parsing '17-05-2016' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Host Since'] = pd.to_datetime(df['Host Since'], infer_datetime_format=True)
C:\Users\sric\AppData\Local\Temp\ipykernel_6968\1651692499.py:17: UserWarning: Parsing '19-08-2014' in DD/MM/YYYY format. Provide format or specify infer_dat

In [36]:
test_df

,Host Since Days,Is Superhost,neighbourhood,Neighborhood Group,Latitude,Longitude,Is Exact Location,Property Type,Room Type,Accomodates,Bathrooms,Bedrooms,Beds,Guests Included,Min Nights,Reviews,Instant Bookable,Has Responses
0,1812.0,t,Kreuzberg,Friedrichshain-Kreuzberg,52.50053,13.44184,t,Apartment,Entire home/apt,2,1,1,1,2,2,131,f,True
1,2768.0,f,Friedrichshain,Friedrichshain-Kreuzberg,52.51647,13.45862,t,Apartment,Entire home/apt,4,1.5,2,4,2,5,22,f,True
2,3100.0,f,Friedrichshain,Friedrichshain-Kreuzberg,52.52481,13.44676,f,Apartment,Private room,2,1,1,2,1,2,20,t,True
3,698.0,f,Tempelhof,Tempelhof - SchÃ¶neberg,52.46009,13.42204,t,Apartment,Private room,3,1,3,4,1,2,19,t,True
4,1494.0,f,Friedrichshain,Friedrichshain-Kreuzberg,52.50516,13.45957,f,Apartment,Private room,2,1,1,1,1,7,3,t,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7837,1914.0,f,Prenzlauer Berg,Pankow,52.54392,13.42416,t,Apartment,Private room,2,2,1,1,1,1,0,f,True
7838,1673.0,f,SchÃ¶neberg,Tempelhof - SchÃ¶neberg,52.48460,13.35230,t,Apartment,Entire home/apt,3,1,1,2,1,7,1,f,True
7839,2426.0,t,Prenzlauer Berg,Pankow,52.54390,13.42881,t,Apartment,Entire home/apt,4,1,1,2,2,3,30,f,True
7840,822.0,f,NeukÃ¶lln,NeukÃ¶lln,52.48783,13.42376,f,Apartment,Entire home/apt,3,1,1,1,1,13,1,f,True


In [33]:
train_df

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0
1,6436842.0,*,5302290.0,Dulie,2013-04-07,NaN,NaN,f,Pankow,Pankow,...,90.0,9.0,9.0,10.0,10.0,9.0,10.0,f,f,41.0
2,10559468.0,*,59151456.0,Geank,2016-02-07,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,50.0
3,27215482.0,*,193452785.0,Alix,2018-06-26,NaN,NaN,f,Friedrichshain,Friedrichshain-Kreuzberg,...,100.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f,50.0
4,27287546.0,*,205870244.0,Lurina,2013-05-16,within a few hours,92%,t,Prenzlauer Berg,Pankow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15687,12936631.0,*,19460983.0,Ascharl,2013-06-22,a few days or more,33%,f,Kreuzberg,Friedrichshain-Kreuzberg,...,98.0,10.0,9.0,10.0,10.0,10.0,10.0,f,f,45.0
15688,8207346.0,*,30411910.0,Marki,2015-04-19,NaN,NaN,f,NeukÃ¶lln,NeukÃ¶lln,...,100.0,10.0,9.0,10.0,10.0,10.0,9.0,f,f,130.0
15689,12246808.0,*,89972306.0,Ada,2016-08-11,NaN,NaN,f,Wedding,Mitte,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,19.0
15690,10886411.0,*,10108952.0,Lanna,2015-12-06,within a day,50%,f,SchÃ¶neberg,Tempelhof - SchÃ¶neberg,...,90.0,10.0,10.0,10.0,10.0,10.0,9.0,f,f,100.0


In [ ]:
train_df.to_csv('test_airbnb_berlin_preprocessed.csv')